# AHRS

This Notebook showcases the most important classes and functions included in the Python package `ahrs`.

Here we will explore the basic use of:

- Class [DCM](https://ahrs.readthedocs.io/en/latest/dcm/classDCM.html)
- Class [Quaternion](https://ahrs.readthedocs.io/en/latest/quaternion/classQuaternion.html)
- Class [QuaternionArray](https://ahrs.readthedocs.io/en/lamy/quaternion/classQuaternionArray.html)
- The new class [Sensors](https://ahrs.readthedocs.io/en/latest/sensors.html) to simulate sensor data.
- The use of [Attitude estimation algorithms](https://ahrs.readthedocs.io/en/latest/filters.html).
- [Metrics functions](https://ahrs.readthedocs.io/en/latest/metrics.html) for orientation representations.
- The [World Magnetic Model](https://ahrs.readthedocs.io/en/latest/wmm.html)
- The [World Geodetic System](https://ahrs.readthedocs.io/en/latest/wgs84.html)
- And diverse tools included in `ahrs`.

### Helping Packages

Plotting and data-handling tools are imported from the script `tools.py` located in the current directory.

- `plot` shows time-series data in vertically stacked plots.
- `plot3` shows a 3D scene, where particles, frames, and items exist and interact in the same space.

Packages `matplotlib` and `ipympl` are required to build interactive visualizations in the Notebook. Make sure you have those installed.

These tools simplify the visualization of orientations in 3d, or time-series data, but are **NOT** included in the `ahrs` package.

Once you have `ahrs` installed (which also installs `numpy`) and you have the forementioned libraries, we can start by setting our notebook up.

In [ ]:
from madgwick_filter import compare, data_processing

In [ ]:
# Use widgets
%matplotlib widget


# Import plotting tools
from madgwick_filter.tools_ahrs import plot
from madgwick_filter.tools_ahrs import plot3
import ahrs
import mrob
import twistnsync as tns
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
plt.rcParams['font.size'] = 14
# Seed random generator
GENERATOR = np.random.default_rng(42)


## Attitude Estimators

Perhaps the most valued contribution of `ahrs` is its collection of attitude estimation algorithms. You can find a list [here](https://ahrs.readthedocs.io/en/lamy/filters.html)

Let's jut explore one famous example: The [Madgwick Filter](https://ahrs.readthedocs.io/en/lamy/filters/madgwick.html).


In [ ]:
x3_path = "madgwick_filter/recordings/X3_simple/Walking_2025-03-06_18-37-32.144_TGW"
# x3_path = "madgwick_filter/recordings/X3_simple/standing_still_2025-03-05_19-11-59.134_TGW"
# x3_path = "madgwick_filter/recordings/X3_simple/basic_motions_2025-03-05_19-05-58.195_TGW"
# x3_path = "madgwick_filter/recordings/X3_simple/walking_2025-03-05_18-54-04.492_TGW"
# x3_path = "madgwick_filter/recordings/X3_simple/walking_talking_2025-03-05_19-01-51.858_TGW"
# x3_path = "madgwick_filter/recordings/X3_simple/Standing_still_2025-03-06_18-50-19.416_TGW"
# x3_path = "madgwick_filter/recordings/X3_simple/Basic_motions_2025-03-06_18-45-09.848_TGW"
# x3_path = "madgwick_filter/recordings/X3_simple/Walking_2025-03-06_18-37-32.144_TGW"
# x3_path = "madgwick_filter/recordings/X3_simple/Walking_talking_2025-03-06_18-41-53.764_TGW"
# x3_path = "madgwick_filter/recordings/X3_simple/Random_walk_2025-03-06_19-02-36.207_TGW"
mocap_path = "madgwick_filter/recordings/Mocap_simple/Walking_Take 2025-03-06 06.38.58 PM.csv"
# mocap_path = "madgwick_filter/recordings/Mocap_simple/standing_still_Take 2025-03-05 06.55.27 PM.csv"
# mocap_path = "madgwick_filter/recordings/Mocap_simple/basic_motions_Take 2025-03-05 06.55.27 PM.csv"
# mocap_path = "madgwick_filter/recordings/Mocap_simple/walking_Take 2025-03-05 06.55.27 PM.csv"
# mocap_path = "madgwick_filter/recordings/Mocap_simple/walking_talking_Take 2025-03-05 06.55.27 PM.csv"
# mocap_path = "madgwick_filter/recordings/Mocap_simple/Standing_still_Take 2025-03-06 06.38.58 PM_003.csv"
# mocap_path = "madgwick_filter/recordings/Mocap_simple/Basic_motions_Take 2025-03-06 06.38.58 PM_002.csv"
# mocap_path = "madgwick_filter/recordings/Mocap_simple/Walking_Take 2025-03-06 06.38.58 PM.csv"
# mocap_path = "madgwick_filter/recordings/Mocap_simple/Walking_talking_Take 2025-03-06 06.38.58 PM_001.csv"
# mocap_path = "madgwick_filter/recordings/Mocap_simple/Random_walk_Take 2025-03-06 06.38.58 PM_004.csv"


# Madgwick filter and game rotation vector comparison

In [ ]:
t_sm, data_gyr, data_acc, data_magn = data_processing.import_combined_data(os.path.join(x3_path, "combined_imu_data_7.csv"))
t_sm = t_sm / 1000                  # ms to s

t_grv, gamerotvec = data_processing.import_gamerotvec_data(os.path.join(x3_path, "game_rotation_vector_12.csv"))
t_grv = t_grv / 1000                  # ms to s

compare.compare_smartphone_to_gamerotvec(t_sm, data_gyr, data_acc, data_magn, t_grv, gamerotvec)

# Madgwick filter and game rotation vector comparison

In [ ]:
data_mocap_t, data_mocap_q, data_mocap_T, data_mocap_Markers_xyz = data_processing.import_mocap_data(mocap_path)

compare.compare_gamerotvec_to_mocap(t_grv, gamerotvec, data_mocap_t, data_mocap_q, 100)

In [ ]:
#data_gyr_t_my, data_gyr_my = data_processing.import_data(os.path.join(x3_path, "gyroscope_3.csv"))
#data_acc_t_my, data_acc_my = data_processing.import_data(os.path.join(x3_path, "accelerometer_1.csv"))
#data_magn_t_my, data_magn_my = data_processing.import_data(os.path.join(x3_path, "magnetic_field_5.csv"))

#downscale = 1
#t_base = data_magn_t_my[::downscale]
#t_base, data_gyr_my_sync, data_acc_my_sync, data_magn_my_sync = data_processing.sync_data(t_base, data_gyr_t_my, data_gyr_my, data_acc_t_my, data_acc_my, data_magn_t_my, data_magn_my)

In [ ]:
t_base, data_gyr, data_acc, data_magn = data_processing.import_combined_data(os.path.join(x3_path, "combined_imu_data_7.csv"))
t_base = t_base/1000

In [ ]:
downscale = 1
t_base, data_gyr, data_acc, data_magn = data_processing.downsample(downscale, t_base, data_gyr, data_acc, data_magn)

In [ ]:
plot(data_gyr)

In [ ]:
print(data_gyr.shape)
print(data_acc.shape)
print(data_magn.shape)
freq_my = 100/downscale # Hz
# if frequency of MoCap is lower than resulting from data - change to MoCap's 240 Hz

Now that we generated IMU data, we can use it to estimate the original attitudes (orientations) with our Madgwick Filter.

In [ ]:
madgwick_MARG = ahrs.filters.Madgwick(gyr=data_gyr,
                                 acc=data_acc,
                                 mag=data_magn,
                                 frequency=freq_my)

madgwick_IMU = ahrs.filters.Madgwick(gyr=data_gyr,
                                 acc=data_acc,
                                 frequency=freq_my)

Done!

The `Madgwick` object uses the given arrays to immediately perform the full computation of the orientations.

These orientations are in an $N\times 4$ array accessible in the attribute called `Q` (stands for Quaternions).

In [ ]:
plot(madgwick_IMU.Q)

Q = [w i j k] - [red green blue gold]

# Here goes comparison with Motion Capture as a reference

In [ ]:
data_mocap_t, data_mocap_q, data_mocap_T, data_mocap_Markers_xyz = data_processing.import_mocap_data(mocap_path)

In [ ]:
data_mocap_t, data_mocap_q, data_mocap_T, data_mocap_Markers_xyz = data_processing.downsample(downscale, data_mocap_t, data_mocap_q, data_mocap_T, data_mocap_Markers_xyz)

In [ ]:
plot(data_mocap_q, madgwick_IMU.Q)

In [ ]:
#t_data_zeroed = (t_base - t_base[0]) / 1000
#t_data_all_sync, data_gyr_sync_mocap_my, data_acc_sync_mocap_my, data_magn_sync_mocap_my, data_quat_R_sync = madgwick_filter.sync_mocap_and_data(data_quat_t, data_quat_R, t_data_zeroed, data_gyr_my_sync, data_acc_my_sync, data_magn_my_sync)
# no more need in sync, because it's done after TwistnSync launch

In [ ]:
i_start = 0
t_base, data_gyr, data_acc, data_magn, data_mocap_t, data_mocap_q = data_processing.arrays_from_i(i_start, t_base, data_gyr, data_acc, data_magn, data_mocap_t, data_mocap_q)

In [ ]:
madgwick_shifted_MARG = ahrs.filters.Madgwick(gyr=data_gyr,
                                 acc=data_acc,
                                 mag=data_magn,
                                 frequency=freq_my)

madgwick_shifted_IMU = ahrs.filters.Madgwick(gyr=data_gyr,
                                 acc=data_acc,
                                 frequency=freq_my)

In [ ]:
plot(madgwick_shifted_MARG.Q)
plot(madgwick_shifted_IMU.Q)

In [ ]:
plot(data_mocap_q, madgwick_shifted_MARG.Q)

# Smartphone and Mocap data sync

More information can be found in the [paper](https://www.mdpi.com/1424-8220/21/1/68).

Here we find time offset and relative transformation between X3 Smartphone's sensors output and Motion Capture system data of tracking the smartphone

## Filter without magnetometer

In [ ]:
time_sync_imu, qsa, qra = compare.compare_smartphone_to_mocap(t_base, madgwick_shifted_IMU.Q, data_gyr,
                                                          data_mocap_t, data_mocap_q, 100)

# Result depends on IMU smoothing window

Walking dataset

For exxample, starting with 5 error in the end is pretty low, but very big in middle. Then, for 10-20 error is quite big. For 40 it's 0.15 and 0.2.  For 70, final error is 0.134 - lowest, and in middle = 0.15. For 100 it's 0.3 and 0.1. For 200 every error grows.

Result - need to find optimal smoothing window. Generally, increasing smoothing we lose final accuracy but increase intermediate accuracy.

## Sometimes filter with magnetometer shows better perfomance

In [ ]:
compare.compare_smartphone_to_mocap(t_base, madgwick_shifted_MARG.Q, data_gyr,
                                                          data_mocap_t, data_mocap_q, 100, gyro=False)      # need to set gyro=False, or the MARG data will not be even used

# Madgwick filter and game rotation vector comparison

In [ ]:
t_sm, data_gyr, data_acc, data_magn = data_processing.import_combined_data(os.path.join(x3_path, "combined_imu_data_7.csv"))
t_sm = t_sm / 1000                  # ms to s

t_grv, gamerotvec = data_processing.import_gamerotvec_data(os.path.join(x3_path, "game_rotation_vector_12.csv"))
t_grv = t_grv / 1000                  # ms to s

compare.compare_smartphone_to_gamerotvec(t_sm, data_gyr, data_acc, data_magn, t_grv, gamerotvec)

# Madgwick filter and game rotation vector comparison

In [ ]:
data_mocap_t, data_mocap_q, data_mocap_T, data_mocap_Markers_xyz = data_processing.import_mocap_data(mocap_path)

compare.compare_gamerotvec_to_mocap(t_grv, gamerotvec, data_mocap_t, data_mocap_q, 100)